In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install funpymodeling

In [ ]:
# Check current working directory.
import os
retval = os.getcwd()
print ("Current working directory %s" % retval)


Current working directory /content


In [ ]:
os.chdir('/content')

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars
from funpymodeling.data_prep import todf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Cargar el dataset

In [ ]:
data=pd.read_csv('/content/sample_data/clase_2_train_data.csv')


# Dividir los datos

In [ ]:
data.head()

,# x1,x2,y
0,-0.128629,0.836679,0.0
1,1.380974,-0.486545,1.0
2,1.282733,0.161911,1.0
3,0.787163,-0.396736,1.0
4,2.024601,-0.521024,1.0


To categorical

In [ ]:
from keras.utils import to_categorical 
data_y = to_categorical(data['y'], num_classes=2)
data_y

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

Normalizar datos

In [ ]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_x = scaler.fit_transform(data.iloc[:,:-1].values)

Separación tr/ts:

In [ ]:
from sklearn.model_selection import train_test_split

x_ntrain, x_ntest, yn_train, yn_test = train_test_split(data_x, data_y, test_size=0.2)
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:,:-1].values, data_y, test_size=0.2)


In [ ]:
x_train.shape

(720, 2)

In [ ]:
y_train.shape

(720, 2)

In [ ]:
x_train.shape
status(x_train)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,0,0,0.0,0,0.0,720,float64
1,1,0,0.0,0,0.0,720,float64


Definir el modelo de la red neuronal en Keras.

In [ ]:
# Carga de libs

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense

In [ ]:
_,in_shape=x_train.shape

In [ ]:
model = Sequential()

model.add(Dense(2, input_dim=in_shape)) # input_dim son las variables de entrada # input + H1
model.add(Dense(3, activation='relu')) # H1
model.add(Dense(2, activation='relu')) # H2
model.add(Dense(2, activation='softmax')) # softmax necesitamos N neuronas

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 6         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 6         
Total params: 29
Trainable params: 29
Non-trainable params: 0
_________________________________________________________________


compilacion


In [ ]:
model.compile(optimizer='RMSprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32) 

Epoch 1/100
23/23 [==============================] - 0s 9ms/step - loss: 0.6742 - accuracy: 0.7764 - val_loss: 0.6628 - val_accuracy: 0.7833
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6519 - accuracy: 0.7806 - val_loss: 0.6408 - val_accuracy: 0.7833
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6286 - accuracy: 0.7931 - val_loss: 0.6169 - val_accuracy: 0.8222
Epoch 4/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6050 - accuracy: 0.8028 - val_loss: 0.5938 - val_accuracy: 0.8167
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 0.5799 - accuracy: 0.8083 - val_loss: 0.5684 - val_accuracy: 0.8167
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 0.5552 - accuracy: 0.8056 - val_loss: 0.5452 - val_accuracy: 0.8167
Epoch 7/100
23/23 [==============================] - 0s 2ms/step - loss: 0.5333 - accuracy: 0.8083 - val_loss: 0.5251 - val_accuracy: 0.8056
Epoch 8/100
2

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_all(history):
  fig1 = go.Figure()
  fig1.add_trace(go.Scattergl(y=history.history['accuracy'], name='Train'))
  fig1.add_trace(go.Scattergl(y=history.history['val_accuracy'], name='Valid'))
  fig1.update_layout(height=300, width=400,xaxis_title='Epoch', yaxis_title='accuracy')

  fig2 = go.Figure()
  fig2.add_trace(go.Scattergl(y=history.history['loss'], name='Train'))
  fig2.add_trace(go.Scattergl(y=history.history['val_loss'], name='Valid'))
  fig2.update_layout(height=300, width=400,xaxis_title='Epoch', yaxis_title='Loss')

  return [fig1,fig2]

In [ ]:
fig1,fig2=plot_all(history)
fig1.show()
fig2.show()

In [ ]:
scores = model.evaluate(x_test, y_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.3174 - accuracy: 0.8611


In [ ]:
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

accuracy: 86.11%
